In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/Users/parthshah/Documents/Northeastern/Spring2022/BigDataAnalytics/Assignment3/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|███████████████████████████| 850M/850M [00:17<00:00, 50.6MB/s]
Downloading: 100%|███████████████████████████| 773k/773k [00:00<00:00, 5.80MB/s]


In [10]:
article = """
A frontal boundary that had been stalled over the mid-Atlantic had lifted north of the region by the morning of 
June 29. Later that day and into the evening, the front once again approached, this time as a strong cold front,
as low pressure tracked through New England and began to intensify offshore in the Gulf of Maine. 
The combination of strong frontal forcing and a warm, unstable environment ahead of the front led to 
widespread severe thunderstorms developing. Numerous reports of damaging wind, as well as some hail, 
were received in association with these storms.
"""

In [11]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

In [12]:
inputs

tensor([[21603,    10,    71,   851,   138, 20430,    24,   141,   118,     3,
          8407,  1361,   147,     8,  2076,    18,   188,    17,  1618,  1225,
           141, 19464,  3457,    13,     8,  1719,    57,     8,  1379,    13,
          1515,  2838,     5, 10511,    24,   239,    11,   139,     8,  2272,
             6,     8,   851,   728,   541, 15319,     6,    48,    97,    38,
             3,     9,  1101,  2107,   851,     6,    38,   731,  1666, 22679,
           190,   368,  2789,    11,  1553,    12,  9608,  4921, 16667,    16,
             8, 13566,    13, 13905,     5,    37,  2711,    13,  1101,   851,
           138, 19060,    11,     3,     9,  1978,     6, 27644,  1164,  2177,
            13,     8,   851,  2237,    12, 14047,  5274, 31250,     7,  2421,
             5, 25638,   302,  2279,    13, 18488,  2943,     6,    38,   168,
            38,   128, 22690,     6,   130,  1204,    16,  6028,    28,   175,
          5536,     7,     5,     1]])

In [13]:
# generate the summarization output
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,     3,     9,   851,    24,   141,   118,     3,  8407,  1361,
           147,     8,  2076,    18,   188,    17,  1618,  1225, 19464,  3457,
            13,     8,  1719,     3,     5,   865,    24,   239,    11,   139,
             8,  2272,     6,     8,   851,   728,   541, 15319,     3,     5,
           731,  1666, 22679,   190,   126,  2789,    11,  1553,    12,  9608,
          4921, 16667,    16,     8, 13566,    13, 13905,     3,     5,     1]])
<pad> a front that had been stalled over the mid-Atlantic lifted north of the region. later that day and into the evening, the front once again approached. low pressure tracked through new England and began to intensify offshore in the Gulf of Maine.</s>


In [ ]:
import json
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoConfig

def encode(tokenizer, question, context):
    """encodes the question and context with a given tokenizer"""
    encoded = tokenizer.encode_plus(question, context)
    return encoded["input_ids"], encoded["attention_mask"]

def decode(tokenizer, token):
    """decodes the tokens to the answer with a given tokenizer"""
    answer_tokens = tokenizer.convert_ids_to_tokens(
        token, skip_special_tokens=True)
    return tokenizer.convert_tokens_to_string(answer_tokens)

def serverless_pipeline(model_path='./model'):
    """Initializes the model and tokenzier and returns a predict function that ca be used as pipeline"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    def predict(question, context):
        """predicts the answer on an given question and context. Uses encode and decode method from above"""
        input_ids, attention_mask = encode(tokenizer,question, context)
        a =  model(torch.tensor(
            [input_ids]), attention_mask=torch.tensor([attention_mask]))
        start_scores = a['start_logits']
        end_scores = a['end_logits']    
        print(start_scores)
        ans_tokens = input_ids[torch.argmax(
            start_scores): torch.argmax(end_scores)+1]
        answer = decode(tokenizer,ans_tokens)
        return answer
    return predict

# initializes the pipeline
question_answering_pipeline = serverless_pipeline()

def handler(event, context):
    try:
        # loads the incoming event into a dictonary
        body = json.loads(event['body'])
        # uses the pipeline to predict the answer
        answer = question_answering_pipeline(question=body['question'], context=body['context'])
        return {
            "statusCode": 200,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True

            },
            "body": json.dumps({'answer': answer})
        }
    except Exception as e:
        print(repr(e))
        return {
            "statusCode": 500,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True
            },
            "body": json.dumps({"error": repr(e)})
        }